## Import Libraries

In [1]:
import pandas as pd
import colorlover as cl
import h5py
import numpy as np
import pickle

import requests
from os.path import exists
import json
import pandas as pd
import pickle

from datetime import datetime

In [2]:
def get_json(permid, access_token = 'YOUR_ACCESS_TOKEN'):
    
    """
    get_json() function sends query to PermID API and returns the result in the json format.
    
    The sample result looks like:
    get_json('4295895317')
    
    {u'result': {u'instruments': {u'entities': [],
                   u'entityType': u'instruments',
                   u'num': 0,
                   u'start': 1,
                   u'total': 0},
                  u'organizations': {u'entities': [{u'@id': u'https://permid.org/1-4295895317',
                     u'hasURL': u'http://www.landmarkmortgages.com',
                     u'orgSubtype': u'Securities Firm',
                     u'organizationName': u'Landmark Mortgages Ltd',
                     u'primaryTicker': u'NRK'}],
                   u'entityType': u'organizations',
                   u'num': 1,
                   u'start': 1,
                   u'total': 1},
                  u'quotes': {u'entities': [],
                   u'entityType': u'quotes',
                   u'num': 0,
                   u'start': 1,
                   u'total': 0}}}
    
    """
    
    url = 'https://api.thomsonreuters.com/permid/search?q=permid:'
    search_query = url + permid

    headers = {'X-AG-Access-Token' : access_token}
   
    response = requests.get(search_query, headers = headers)

    print 'status code: %s' % response.status_code
    
    if (response.status_code == 429) | (response.status_code == 500):
        print "Retrying..."
        time.sleep(1.0)
        return get_json(permid)
    else:
        content = response.content
        json_content = json.loads(content)
        return json_content

In [3]:
def get_entity_info(permid):
    
    """
    get_entity_info() function consumes PermID and
    returns the associated entity name and ticker information of the input permid
    """
    
    if not isinstance(permid, basestring):
        permid = str(permid)
    
    entity_info = get_json(permid)
    
    if entity_info == {}:
        return []
    
    organization_list = entity_info['result']['organizations'][u'entities']
    
    
    """
    When we first identified the PermID of each entity, we went through the process of finding the most parent company of the mentioned entity.
    So this will return only one result per permid, but we will loop through the list just to ensure that we capture all variations.
    """
    
    entity = organization_list[0].get('organizationName')
    ticker = organization_list[0].get('primaryTicker')

    return entity, ticker

In [4]:
def load_data():
    """
    
    """
    
    permid_to_name_path = './permid_to_name.pkl'
    permid_to_ticker_path = './permid_to_ticker.pkl'
    df_path = './predicted_df_final.pkl'
    
    if exists(permid_to_ticker_path) and exists(permid_to_name_path):
        permid_to_name = pickle.load(open(permid_to_name_path, 'rb'))
        permid_to_ticker = pickle.load(open(permid_to_ticker_path, 'rb'))
        
    else:
        permids = pickle.load(open('./all_rel_sent_permid.pkl','rb'))
        permids = list(set([str(permid) for permid in permids]))

        permid_dict = {permid: (get_entity_info(permid)) for permid in permids}

        permid_to_name = {permid: v[0] for permid, v in permid_dict.items()}
        permid_to_ticker = {permid: v[1] for permid, v in permid_dict.items()}

        pickle.dump(permid_to_name, open(permid_to_name_path, 'wb'))
        pickle.dump(permid_to_ticker, open(permid_to_ticker_path, 'wb'))
        
    if exists(df_path):
        df = pickle.load(open(df_path, 'rb'))
        
        return df, permid_to_name, permid_to_ticker
        
    else:
        df = pickle.load(open('./predicted_df.pkl','rb'))
        df['permid'] = df.permid.apply(str)
        df['entity_name'] = df.permid.map(permid_to_name)
        df['date_formatted'] = df.date.apply(lambda x: datetime.strptime(x, '%Y%m%d'))
        df['date_month'] = df.date_formatted.apply(lambda d: d.to_period('M'))

        reduced_df = df[['date_formatted', 'date_month', 'permid', 'entity_name', 'headline', 'txt', 'relevant_sent_list', 'predictions', 'distress_signal']]
        reduced_df.columns = ['date', 'date_month', 'permid', 'entity', 'headline', 'txt', 'sent_list', 'predictions', 'distress_signal']
        
        reduced_df = reduced_df.sort_values(['date','permid'], ascending = True)
        reduced_df = reduced_df.reset_index(drop = True)
        
        daily_signal = reduced_df.groupby(['date','permid'], as_index=False)['distress_signal'].mean()
        daily_signal.columns = ['date','permid','daily_avg']
        
        monthly_signal = reduced_df.groupby(['date_month','permid'], as_index=False)['distress_signal'].mean()
        monthly_signal.columns = ['date_month','permid','monthly_avg']
        
        reduced_df = pd.merge(reduced_df, daily_signal, on = ['date','permid'], how="left")
        reduced_df = pd.merge(reduced_df, monthly_signal, on = ['date_month','permid'], how="left")
        reduced_df = reduced_df.reset_index(drop = True)
        
        pickle.dump(reduced_df, open(df_path,'wb'))
        
        return reduced_df, permid_to_name, permid_to_ticker

In [5]:
df, permid_to_name, permid_to_ticker = load_data()

In [6]:
df.head()

,date,date_month,permid,entity,headline,txt,sent_list,predictions,distress_signal,daily_avg,monthly_avg
0,2007-07-31,2007-07,4295895317,Landmark Mortgages Ltd,Lloyds divi cheer as profits leap to £2bn,"BRITAINS fifth-largest bank, Lloyds_TSB, today...",[The banks' half-year haul Bank Profit (£mil...,[0.190399],0.190399,0.095203,0.095203
1,2007-07-31,2007-07,4295895317,Landmark Mortgages Ltd,TAKING STOCK,Lehman Brothers cut Northern_Rock from overwei...,[Lehman Brothers cut Northern_Rock from overwe...,[5.94149e-06],0.000006,0.095203,0.095203
2,2007-08-02,2007-08,4295895317,Landmark Mortgages Ltd,Street Talk,Reshuffling of funds management decks Marcus F...,[Other UK banks that have been linked with NAB...,[0.0404233],0.040423,0.040423,0.027021
3,2007-08-04,2007-08,4295895317,Landmark Mortgages Ltd,Europe succumbs,2007-08-04 The European ABS markets might just...,"[Northern_Rock, for example, has a tradition o...",[0.000421468],0.000421,0.000421,0.027021
4,2007-08-07,2007-08,4295895317,Landmark Mortgages Ltd,TAKING STOCK,BANKING AND FINANCE Credit Suisse has cuts its...,[BANKING AND FINANCE Credit Suisse has cuts it...,[2.89664e-05],0.000029,0.005383,0.027021


### PermID to Bank Name and Ticker

Load the list of all PermID's

In [7]:
permids = pickle.load(open('./all_rel_sent_permid.pkl','rb'))

Since there are duplicate entries, we will need to list(set())

In [8]:
permids = list(set([str(permid) for permid in permids]))

In [9]:
permids

['4295895343',
 '4295884647',
 '4297588026',
 '4295903783',
 '8589934337',
 '4296717582',
 '5000004470',
 '5000084509',
 '4295870355',
 '4297735137',
 '4295868946',
 '4295866136',
 '8589934326',
 '4295870762',
 '4295884483',
 '4295874912',
 '5037277329',
 '4298052532',
 '4295875749',
 '4296034228',
 '4295870369',
 '5000072598',
 '4295870163',
 '5000003784',
 '5035718674',
 '8589934212',
 '4295870566',
 '4295884931',
 '4295895317',
 '4295869051',
 '4295895499',
 '5001167863',
 '4295885329',
 '4295889322',
 '4295886766',
 '5000061789',
 '4295912046',
 '8589934262',
 '4295869477',
 '5000793106',
 '4295865936',
 '5000075035',
 '5000035946',
 '4296729968',
 '4297177260',
 '4295872054',
 '4295870015',
 '4295926398',
 '5035414638',
 '4295869198',
 '5036183005',
 '8589934173',
 '5000062202',
 '4296201178',
 '4295872108',
 '5000277648',
 '5001440324',
 '4296555134',
 '5000718498',
 '8589934296',
 '4295886830',
 '5036161594',
 '4297304387',
 '4295866161',
 '4295895363',
 '4295870630',
 '42965551

In [10]:
permid_dict = {permid: (get_entity_info(permid)) for permid in permids}

status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 200
status code: 2

In [11]:
permids_to_names = {permid: v[0] for permid, v in permid_dict.items()}
permids_to_tickers = {permid: v[1] for permid, v in permid_dict.items()}

In [12]:
# pickle.dump(permids_to_names,open('./permid_to_name.pkl', 'wb'))
# pickle.dump(permids_to_tickers,open('./permid_to_ticker.pkl', 'wb'))

In [13]:
df = pickle.load(open('./predicted_df.pkl','rb'))
df.head()

,date,permid,entities,headline,txt,relevant_sent_list,predictions,distress_signal,sent_count
0,20090930,4295859079,[Volksbanken],VEB to raise loan from German banks to finance...,MOSCOW. (Interfax) - Vnesheconombank (VEB) has...,[DZ Bank is part of the FinanzVerbund cooperat...,[3.64003e-05],0.000036,1
1,20121113,4295865896,[Tønder_Bank_A/S],Report: Accountancy firms probed over Tønder B...,The Danish Business Authority launched an inve...,[The Danish Business Authority launched an inv...,[8.59119e-05],0.000086,1
2,20121115,4295865896,[Tønder_Bank],DK-Tønder: banking services,Document number - E361909-2012 Number of Offic...,[1) Type procedure Forhandling uden offentlig...,"[0.00191777, 0.00179588]",0.001857,2
3,20121214,4295865896,[Toender_Bank],A still fragile banking sector,"A series of government rescue packages, alongs...",[It is therefore not yet clear that the risk o...,[0.00440103],0.004401,1
4,20111010,4295866136,[Max_Bank],Danish Govt likely to take over Max Bank,COPENHAGEN: The Danish government's winding-up...,[In its annual report published on February 28...,"[0.0184578, 0.0310413, 0.00383605, 2.90744e-06...",0.006022,9


In [14]:
df['permid'] = df.permid.apply(str)

In [15]:
df['entity_name'] = df.permid.map(permids_to_names)

In [16]:
df.head()

,date,permid,entities,headline,txt,relevant_sent_list,predictions,distress_signal,sent_count,entity_name
0,20090930,4295859079,[Volksbanken],VEB to raise loan from German banks to finance...,MOSCOW. (Interfax) - Vnesheconombank (VEB) has...,[DZ Bank is part of the FinanzVerbund cooperat...,[3.64003e-05],0.000036,1,immigon portfolioabbau AG
1,20121113,4295865896,[Tønder_Bank_A/S],Report: Accountancy firms probed over Tønder B...,The Danish Business Authority launched an inve...,[The Danish Business Authority launched an inv...,[8.59119e-05],0.000086,1,TOENDER BANK under konkurs A/S
2,20121115,4295865896,[Tønder_Bank],DK-Tønder: banking services,Document number - E361909-2012 Number of Offic...,[1) Type procedure Forhandling uden offentlig...,"[0.00191777, 0.00179588]",0.001857,2,TOENDER BANK under konkurs A/S
3,20121214,4295865896,[Toender_Bank],A still fragile banking sector,"A series of government rescue packages, alongs...",[It is therefore not yet clear that the risk o...,[0.00440103],0.004401,1,TOENDER BANK under konkurs A/S
4,20111010,4295866136,[Max_Bank],Danish Govt likely to take over Max Bank,COPENHAGEN: The Danish government's winding-up...,[In its annual report published on February 28...,"[0.0184578, 0.0310413, 0.00383605, 2.90744e-06...",0.006022,9,MAX Bank A/S


In [17]:
from datetime import datetime

In [18]:
df['date_formatted'] = df.date.apply(lambda x: datetime.strptime(x, '%Y%m%d'))

In [19]:
df2 = df[['date_formatted', 'permid', 'entity_name', 'headline', 'txt', 'relevant_sent_list', 'predictions', 'distress_signal']]

In [20]:
df2.columns = ['date', 'permid', 'entity', 'headline', 'txt', 'sent_list', 'predictions', 'distress_signal']

In [21]:
df2 = df2.sort_values(['date','permid'], ascending = True)
df2 = df2.reset_index(drop = True)

In [22]:
df2['date_month'] = df2.date.apply(lambda d: d.to_period('M'))

In [23]:
df2['distress_signal'] = df2.distress_signal.round(4)

In [24]:
daily_signal = df2.groupby(['date','permid'], as_index=False)['distress_signal'].max().round(4)
daily_signal.columns = ['date','permid','daily_max']

In [25]:
monthly_signal = df2.groupby(['date_month','permid'], as_index=False)['distress_signal'].max().round(4)
monthly_signal.columns = ['date_month','permid','monthly_max']

In [26]:
df2 = pd.merge(df2, daily_signal, on = ['date','permid'], how="left")

In [27]:
df2 = pd.merge(df2, monthly_signal, on = ['date_month','permid'], how="left")

In [28]:
df2.head()

,date,permid,entity,headline,txt,sent_list,predictions,distress_signal,date_month,daily_max,monthly_max
0,2007-07-31,4295895317,Landmark Mortgages Ltd,Lloyds divi cheer as profits leap to £2bn,"BRITAINS fifth-largest bank, Lloyds_TSB, today...",[The banks' half-year haul Bank Profit (£mil...,[0.190399],0.1904,2007-07,0.1904,0.1904
1,2007-07-31,4295895317,Landmark Mortgages Ltd,TAKING STOCK,Lehman Brothers cut Northern_Rock from overwei...,[Lehman Brothers cut Northern_Rock from overwe...,[5.94149e-06],0.0000,2007-07,0.1904,0.1904
2,2007-08-02,4295895317,Landmark Mortgages Ltd,Street Talk,Reshuffling of funds management decks Marcus F...,[Other UK banks that have been linked with NAB...,[0.0404233],0.0404,2007-08,0.0404,0.3848
3,2007-08-04,4295895317,Landmark Mortgages Ltd,Europe succumbs,2007-08-04 The European ABS markets might just...,"[Northern_Rock, for example, has a tradition o...",[0.000421468],0.0004,2007-08,0.0004,0.3848
4,2007-08-07,4295895317,Landmark Mortgages Ltd,TAKING STOCK,BANKING AND FINANCE Credit Suisse has cuts its...,[BANKING AND FINANCE Credit Suisse has cuts it...,[2.89664e-05],0.0000,2007-08,0.0107,0.3848


In [30]:
pickle.dump(df2, open('./data.pkl', 'wb'))